In [1]:
from docking.dock import MolDox
from pymol import cmd
from rdkit import Chem
from docking.utils import split_pdb
# from docking.viewer import InteractionMap, Mol3D
import random

## 1. Define your output folder and some filenames

In [2]:
output_dir = 'test_output'  # This directory is going to hold all output and intermediate files
input_file = None  # In this example we are fetching a pdb from the internet

receptor_file = f'{output_dir}/1AZ8_clean.pdb'  # This name will be used to save the cleand pdb to
ref_ligand_file = f'{output_dir}/1AZ8_lig.mol2'  # This name will be used to save the cleand reference ligand to


# File of the ligands you want to dock. This file could be any mol2 or sdf file
docking_ligands = f'{output_dir}/1AZ8_lig.mol2'  # In this example we're re-docking the ligand we took from the pdb

## 2. Split a pdb of a receptor+ligand
You can also skip this step, do it manually, and just supply those filenames in the next step

In [3]:
# We are fetching a pdb from the internet. 
# If you have your own file, set fetch=None and give your file in infile='path/to/your/file.pdb'
split_pdb(receptor_file, ref_ligand_file, fetch='1AZ8', infile=None)  

Feature has expired.
Feature:       PYMOL_MAIN
Expire date:   01-feb-2022
License path:  /Users/derek/.pymol/license.lic:/Library/Application Support/Schrodinger/licenses:
FlexNet Licensing error:-10,32


## 3. Prepare the receptor for docking


In [4]:
dox = MolDox(output_dir)  # Initiate the MolDox class that is going to do the heavy lifting
dox.prep_receptor(receptor_file, ref_ligand_file, ph=7.4, keep_water=True, renumber=True)

Cleanup pdb (replacing noncanonical residues and keeping waters
Adding missing atoms if there are any
Adding missing hydrogens (pH=7.4)


Renumbering residues
Convert input pdb to pdbqt.
Make protein rigid.
Preserve hydrogens
Preserve atom indices
Computing partial charges


*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 1AZ8)



In [5]:
# Check out how the receptor and ligand looks (or don't)
dox.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## 4. Prepare all ligands you are doing to dock
Hand over the file (we defined the file path in the docking_ligands variable) that holds all the ligands
If you're using a different format (like .sdf), you should specify it. Only .mol2 and .sdf are tested,
but more formats are technically supported.

Gasteigar charges are always calculated, missing hydrogens are always added. However, this function has a few extra arguments to customize the ligand prep. By default they are set like this:

hydrate=False  # hydrate molecule by building waters around it
keep_nonpolar_hydrogens=False
pH_value=None  # Correct for a specific pH (takes a float, e.g., 7.4)
make3d=True  # If MolDox detects that all z-coordinates are 0, it will make it 3d
remake3d=False  # If you set this to True, 3D coordinates of all ligands will be recomputed
forcefield='mmff94'  # This forcefield is used to make molecules 3d. supports: gaff, ghemical, mmff94, mmff94s, uff
steps_3d=50  # n steps used for computing 3d coordinates
steps_3d_localopt=500  # n steps of local optimization of 3d coordinates

In [6]:
dox.prep_ligands(docking_ligands, format='mol2')

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is 1AZ8)



## 5. Perform docking using Autodock Vina.
THis function takes the arguments: box_center and box_size. The are set to None by default, which will result in MolDox calculating the box dimensions autmatically. 
You can do it yourself by first running dox.find_box('receptor_file.pdb', 'ligand_file.mol2', box_extension=5) or supplying your own custom box dimensions (see documentations)

By default autodock_all() uses the following parameters:

box_extension=5  # extend the box of the refence ligand by n Angstrom
exhaustiveness=10  # n steps of random perturbation of the conformation and optimization aka the search space
n_poses=10  # generate n poses per ligand
scoring_function='vina'
cpu_cores=0  # 0 means use all cores
seed=42  # random seed to let you replicate random processes
receptor_pdbqt=None
box_center=None
box_size=None

In [7]:
# Docking results in .pdbqt and .sdf format are saved in <your_output_directory>/docking_results
# They are also saved in a dictionary in the MolDox object: dox.docking_results = {'mol_0': 'path.sdf', ...}
dox.autodock_all()

Docking 1 ligands
Computing Vina grid ... done.
Performing docking (random seed: 42) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


## 6. Analyse the results
You can visualize the docked ligand in 3D

In [8]:
dox.viewer_add_ligand('mol_0')
dox.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol